# Accuracy and Speed: `NNdex` vs NumPy

Compares top-k results and latency for exact cosine search between `nndex.NNdex` and a NumPy baseline.


In [1]:
import time
import numpy as np

from nndex import NNdex


In [2]:
rng = np.random.default_rng(7)
rows, dims, k = 50_000, 128, 20
matrix = rng.normal(size=(rows, dims)).astype(np.float32)
query = rng.normal(size=(dims,)).astype(np.float32)


In [3]:
def numpy_topk_cosine(matrix: np.ndarray, query: np.ndarray, k: int) -> tuple[np.ndarray, np.ndarray]:
    matrix_norm = matrix / np.maximum(np.linalg.norm(matrix, axis=1, keepdims=True), 1e-12)
    query_norm = query / max(float(np.linalg.norm(query)), 1e-12)
    scores = matrix_norm @ query_norm
    top_idx = np.argpartition(scores, -k)[-k:]
    ordered = top_idx[np.argsort(scores[top_idx])[::-1]]
    return ordered, scores[ordered]

def benchmark_nndex(matrix: np.ndarray, query: np.ndarray, k: int, repeats: int = 5) -> tuple[np.ndarray, np.ndarray, float]:
    model = NNdex(matrix, normalized=False, backend="cpu", enable_cache=False)
    timings = []
    out_idx = out_scores = None
    for _ in range(repeats):
        t0 = time.perf_counter()
        out_idx, out_scores = model.search(query, k=k)
        timings.append((time.perf_counter() - t0) * 1e3)
    assert out_idx is not None and out_scores is not None
    return out_idx, out_scores, float(np.median(timings))

def benchmark_numpy(matrix: np.ndarray, query: np.ndarray, k: int, repeats: int = 5) -> tuple[np.ndarray, np.ndarray, float]:
    timings = []
    out_idx = out_scores = None
    for _ in range(repeats):
        t0 = time.perf_counter()
        out_idx, out_scores = numpy_topk_cosine(matrix, query, k)
        timings.append((time.perf_counter() - t0) * 1e3)
    assert out_idx is not None and out_scores is not None
    return out_idx, out_scores, float(np.median(timings))


In [4]:
nndex_idx, nndex_scores, nndex_ms = benchmark_nndex(matrix, query, k)
np_idx, np_scores, np_ms = benchmark_numpy(matrix, query, k)

overlap = len(set(nndex_idx.tolist()) & set(np_idx.tolist())) / k
score_mae = float(np.mean(np.abs(nndex_scores - np_scores)))

print(f"NNdex median latency: {nndex_ms:.2f} ms")
print(f"NumPy median latency: {np_ms:.2f} ms")
print(f"Top-k overlap: {overlap:.3f}")
print(f"Score MAE: {score_mae:.6f}")


NNdex median latency: 0.31 ms
NumPy median latency: 3.96 ms
Top-k overlap: 1.000
Score MAE: 0.000000
